In [7]:
#can we build a python via PubMed so that we can provide a direct option to bulk-extract literature texts into an Excel file?

In [8]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
import requests
from bs4 import BeautifulSoup

# Check if requests is installed
try:
    import requests
except ImportError:
    print("The 'requests' library is not installed.")
else:
    print("The 'requests' library is installed.")

# Check if beautifulsoup4 is installed
try:
    from bs4 import BeautifulSoup
except ImportError:
    print("The 'beautifulsoup4' library is not installed.")
else:
    print("The 'beautifulsoup4' library is installed.")

The 'requests' library is installed.
The 'beautifulsoup4' library is installed.


In [11]:
import requests
from bs4 import BeautifulSoup

search_url = 'https://pubmed.ncbi.nlm.nih.gov/?term='
search_term = '(ex-gaussian[Title/Abstract]) AND (reaction time[Title/Abstract])'
results_per_page = 10  # Number of results per page
num_pages = 6  # Number of pages to retrieve (example: 6 pages * 10 results per page = 60 results)

results = []

for page in range(1, num_pages + 1):
    # Send a request to PubMed for each page
    response = requests.get(search_url + search_term + f'&page={page}')
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the search results from the current page
    page_results = soup.find_all('div', class_='docsum-content')
    
    # Add the page_results to the main results list
    results.extend(page_results)

print(len(results))  # Print the total number of search results retrieved

60


In [13]:
titles, authors, years, abstracts, links = [], [], [], [], []

In [21]:
min_length = min(len(titles), len(authors), len(years), len(abstracts))

titles = titles[:min_length]  
authors = authors[:min_length]
years = years[:min_length]
abstracts = abstracts[:min_length] 


In [22]:
import re

In [23]:

for result in results:

  title = result.find('a', class_='docsum-title').text
  titles.append(title)

  author = result.find('span', class_='docsum-authors').text
  authors.append(author)

  year = re.search(r'\d{4}', result.find('span', class_='docsum-journal-citation').text).group()
  years.append(year)

  link = result.find('a', class_='docsum-title')['href']
  links.append(link)

  page = requests.get(f"https://pubmed.ncbi.nlm.nih.gov{link}")
  soup = BeautifulSoup(page.content, 'html.parser')
  abstract = soup.find('div', class_='abstract-content').text
  abstracts.append(abstract)

min_len = min(len(titles), len(authors), len(years), len(abstracts), len(links))  

# Truncate lists

titles = titles[:min_len]
authors = authors[:min_len]
years = years[:min_len]
abstracts = abstracts[:min_len]
links = links[:min_len]

In [25]:
df = pd.DataFrame({
    'Title': titles,
    'Author': authors,
    'Year': years,
    'Abstract': abstracts,
    'Link': links
})

In [26]:
df.to_excel('search_results.xlsx', index=False)